In [2]:
from __future__ import division
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score,accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import Imputer


pd.set_option('display.max_columns',500)
pd.set_option('display.max_colwidth',-1)
%matplotlib inline


In [3]:
df = pd.read_csv('C:/Users/cody8/Desktop/Documentos/Escuela/AMV/parcial 3 y 4/entrenar2.csv')

In [4]:
df.head()

,Cuadrante,c_Total de crimenes,c_HOMICIDIO_DOLOSO,c_LESIONES_POR_ARMA_DE_FUEGO,c_ROBO_A_BORDO_DE_METRO_CV,c_ROBO_A_BORDO_DE_MICROBUS_CV,c_ROBO_A_BORDO_DE_MICROBUS_SV,c_ROBO_A_BORDO_DE_TAXI_CV,c_ROBO_A_CASA_HABITACION_CV,c_ROBO_A_CUENTAHABIENTE_CV,c_ROBO_A_NEGOCIO_CV,c_ROBO_A_REPARTIDOR_CV,c_ROBO_A_REPARTIDOR_SV,c_ROBO_A_TRANSEUNTE_CV,c_ROBO_A_TRANSEUNTE_SV,c_ROBO_A_TRANSPORTISTA_CV,c_ROBO_A_TRANSPORTISTA_SV,c_ROBO_DE_VEHICULO_AUTOMOTOR_CV,c_ROBO_DE_VEHICULO_AUTOMOTOR_SV,c_VIOLACION,obj
0,1,205,6,1,0,19,0,1,4,1,24,4,1,29,2,1,0,48,62,2,0
1,2,41,1,0,0,1,1,1,1,0,13,0,1,2,0,0,0,9,10,1,0
2,3,140,1,3,0,3,0,1,4,4,15,9,3,34,0,0,0,17,46,0,0
3,4,107,10,5,0,2,0,0,4,0,4,20,3,32,1,0,0,10,12,4,0
4,5,109,2,1,0,3,0,2,1,1,19,6,1,16,1,3,0,21,30,2,0


In [5]:
var_disc = [x for x in df.columns if x.startswith('d_')]
var_cont = [x for x in df.columns if x.startswith('c_')]
# obj = d_ROBO_A_BORDO_DE_METRO_SV Jul Ag Sept
X = df[var_cont+var_disc].copy()
y = df['obj'].copy()

print len(var_cont)
print len(var_disc)

19
0


In [6]:
Xt,Xv,yt,yv = train_test_split(X,y,train_size=0.7)

print Xt.shape
print Xv.shape
print yt.shape
print yv.shape

(415, 19)
(178, 19)
(415L,)
(178L,)


In [7]:
sc = MinMaxScaler()
sc.fit(X)
Xt = sc.transform(Xt)
Xv = sc.transform(Xv)

In [8]:
svm = SVC()
#help(SVC)

In [9]:
knl = ['linear', 'poly', 'rbf', 'sigmoid']
dfs=['ovo','ovr']
deg = range(2,6)
param_grid=dict(decision_function_shape=dfs,kernel=knl,degree=deg)

grid=GridSearchCV(svm,param_grid,cv=10,scoring='roc_auc',n_jobs=-1)#,n_iter=10)
#probability = [True]

In [10]:
dicc = dict(kernel = ['linear', 'poly', 'rbf', 'sigmoid'],
degree = range(2,6),
probability = [True])
rsgrid = RandomizedSearchCV(param_distributions=dicc,n_iter=6,
                          cv=3,n_jobs=-1, 
                    estimator=svm,
                    scoring='roc_auc')

In [11]:
%%time
rsgrid.fit(Xt,yt)

Wall time: 1min 54s


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
          fit_params={}, iid=True, n_iter=6, n_jobs=-1,
          param_distributions={'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'probability': [True], 'degree': [2, 3, 4, 5]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score=True, scoring='roc_auc', verbose=0)

In [12]:
#%%time
#grid.fit(Xt,yt)

#print grid.best_params_
#print grid.best_estimator_
#print grid.best_score_                    


In [13]:
print rsgrid.best_params_
print rsgrid.best_estimator_
print rsgrid.best_score_  

{'kernel': 'sigmoid', 'degree': 5, 'probability': True}
SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=5, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.844516370567


In [14]:
svm = rsgrid.best_estimator_

In [15]:
svm.fit(Xt,yt)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=5, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [16]:
print roc_auc_score(y_score=svm.predict_proba(Xt)[:,1],y_true=yt)
print roc_auc_score(y_score=svm.predict_proba(Xv)[:,1],y_true=yv)

0.882652588682
0.895547945205


In [17]:
pd.Series(svm.predict_proba(Xv)[:,1])

0      0.338955
1      0.114210
2      0.110053
3      0.090581
4      0.155490
5      0.073804
6      0.055284
7      0.130583
8      0.891962
9      0.086308
10     0.070334
11     0.062262
12     0.090055
13     0.111463
14     0.142997
15     0.129126
16     0.096898
17     0.048531
18     0.433287
19     0.107668
20     0.096655
21     0.121320
22     0.104420
23     0.099372
24     0.127113
25     0.062950
26     0.040758
27     0.073120
28     0.065095
29     0.079824
         ...   
148    0.043454
149    0.089480
150    0.058765
151    0.080506
152    0.136203
153    0.085126
154    0.226509
155    0.129215
156    0.119266
157    0.093427
158    0.035553
159    0.079153
160    0.018014
161    0.061150
162    0.076272
163    0.080110
164    0.054659
165    0.074569
166    0.081068
167    0.082588
168    0.076303
169    0.125682
170    0.174072
171    0.232103
172    0.104785
173    0.947130
174    0.114093
175    0.037831
176    0.194188
177    0.084353
dtype: float64

In [18]:
vf = pd.read_csv('C:/Users/cody8/Desktop/Documentos/Escuela/AMV/parcial 3 y 4/validar2.csv')

In [19]:
tf = pd.read_csv('C:/Users/cody8/Desktop/Documentos/Escuela/AMV/parcial 3 y 4/entrenar2.csv')

In [20]:
var_disc = [x for x in tf.columns if x.startswith('d_')]
var_cont = [x for x in tf.columns if x.startswith('c_')]
## obj = d_ROBO_A_BORDO_DE_METRO_SV Jul Ag Sept
Xw = tf[var_cont+var_disc].copy()
yw = tf['obj'].copy()



var_discf = [x for x in vf.columns if x.startswith('d_')]
var_contf = [x for x in vf.columns if x.startswith('c_')]
## obj = d_ROBO_A_BORDO_DE_METRO_SV Jul Ag Sept
Xv = vf[var_contf+var_discf].copy()


In [21]:
print Xw.shape
print Xv.shape
print yw.shape

(593, 19)
(199, 19)
(593L,)


In [22]:
Xw = sc.transform(Xw)
Xv = sc.transform(Xv)

In [23]:
svm.fit(Xw,yw)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=5, gamma='auto', kernel='sigmoid',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [24]:
print roc_auc_score(y_score=svm.predict_proba(Xw)[:,1],y_true=yw)

0.888494623656


In [25]:
print accuracy_score(y_pred=svm.predict(Xw),y_true=yw)

0.843170320405


In [27]:
predsvm=pd.Series(svm.predict_proba(Xv)[:,1])
predsvm

0      0.037015
1      0.055886
2      0.068326
3      0.999982
4      0.199316
5      0.094189
6      0.110468
7      0.371806
8      0.072208
9      0.060915
10     0.018151
11     0.024345
12     0.102197
13     0.039654
14     0.054691
15     0.178103
16     0.584508
17     0.037312
18     0.084236
19     0.121867
20     0.024377
21     0.087269
22     0.081067
23     0.144897
24     0.052669
25     0.081957
26     0.065637
27     0.056623
28     0.033317
29     0.207261
         ...   
169    0.049357
170    0.065182
171    0.006583
172    0.059508
173    0.036549
174    0.015152
175    0.067314
176    0.023319
177    0.042607
178    0.159370
179    0.037885
180    0.110408
181    0.149201
182    0.082618
183    0.094576
184    0.038183
185    0.039293
186    0.806450
187    0.042241
188    0.042276
189    0.155407
190    0.026227
191    0.013724
192    0.307927
193    0.308099
194    0.091684
195    0.167195
196    0.281470
197    1.000000
198    0.073983
dtype: float64

In [28]:
predsvm.to_csv('s_311093416_p5.cvs')